In [1]:
%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import gridspec
import matplotlib.patheffects as path_effects

import StringIO
import numpy as np

import imp
bt = imp.load_source('baltic', '/Users/evogytis/Documents/baltic/baltic.py')

import treetime

typeface='Helvetica Neue'

mpl.rcParams['font.weight']=300
mpl.rcParams['axes.labelweight']=300
mpl.rcParams['font.family']=typeface
mpl.rcParams['font.size']=22

masked_set=[]
for line in open('/Users/evogytis/Documents/genomic-horizon/data/masked_set.txt','r'):
    masked_set.append(line.strip('\n'))

print 'Done!'

Done!


Use RAxML to infer ML phylogenies for genome and GP.

In [2]:
%%bash

cd /Users/evogytis/Documents/genomic-horizon/data/alignments/

alns=*.fasta
echo $alns

for aln in $alns
do
    echo $aln
    
    partition=${aln/.fasta/.part} ## partition designations for each alignment
    out=${aln/.fasta/.raxml.tree} ## output format
    
    raxml -f d -T 2 -m GTRCAT -q $partition -c 25 -p 666 -n $out --silent -s $aln ## run raxml
    
done;

trees=RAxML_bestTree*
for tree in $trees
do
    mv $tree /Users/evogytis/Documents/genomic-horizon/data/trees/${tree/RAxML_bestTree./} ## simplify name of best tree
done;

Makona_600_genome.fasta Makona_600_gp.fasta
Makona_600_genome.fasta

RAxML can't, parse the alignment file as phylip file 
it will now try to parse it as FASTA file


Just in case you might need it, a mixed model file with 
model assignments for undetermined columns removed is printed to file Makona_600_genome.part.reduced

IMPORTANT WARNING: Alignment validation warnings have been suppressed. Found 16 duplicate sequences and 21 undetermined columns

Just in case you might need it, an alignment file with 
sequence duplicates and undetermined columns removed is printed to file Makona_600_genome.fasta.reduced

Alignment has 21 completely undetermined sites that will be automatically removed from the input data


This is the RAxML Master Pthread

This is RAxML Worker Pthread Number: 1


This is RAxML version 8.2.8 released by Alexandros Stamatakis on March 23 2016.

With greatly appreciated code contributions by:
Andre Aberer      (HITS)
Simon Berger      (HITS)
Alexey Kozlov     (HITS)
K

Remove RAxML polytomies.

In [2]:
for gene in ['genome','gp']:
    tree_path='/Users/evogytis/Documents/genomic-horizon/data/trees/Makona_600_%s.raxml.tree'%(gene)
    
    ll=bt.tree()
    bt.make_tree(open(tree_path,'r').readline(),ll)
    ll.traverse_tree()
    ll=ll.collapseBranches(lambda x:x.length<=1e-5)
    ll.treeStats()
    
    out_tree=open(tree_path,'w')
    out_tree.write('%s\n'%(ll.toString(numName=True)))
    out_tree.close()


Tree height: 0.001773
Tree length: 0.068033

Numbers of objects in tree: 809 (209 nodes and 600 leaves)


Tree height: 0.003497
Tree length: 0.079726

Numbers of objects in tree: 641 (41 nodes and 600 leaves)



Infer time trees with TreeTime, save time trees and reconstructed ancestral sequences.

In [3]:
root_path='/Users/evogytis/Documents/genomic-horizon/data/'
trees={}
tip_dates={}
for gene in ['genome','gp']:
    tree_path='/Users/evogytis/Documents/genomic-horizon/data/trees/Makona_600_%s.raxml.tree'%(gene)
    
    ll=bt.tree()
    bt.make_tree(open(tree_path,'r').readline(),ll)
    tip_dates[gene]={k.numName:bt.decimalDate(k.numName.split('|')[-1]) if k.numName not in masked_set else (2013.91506849,2015.91506849) for k in ll.Objects if k.branchType=='leaf'}
    
    tt = treetime.TreeTime(dates=tip_dates[gene], tree=root_path+'trees/Makona_600_%s.raxml.tree'%(gene), aln=root_path+'alignments/Makona_600_%s.fasta'%(gene), gtr='HKY85')
    trees[gene]=tt
    
    new_root=tt.reroot_to_best_root()
    new_root.mutation_length=0.0
    tt.reroot(new_root)
    treetime.treetime.treetime_to_newick(tt,root_path+'trees/Makona_600_%s.rooted.tree'%(gene)) ## save tree
    
    tt.run(root='best', infer_gtr=False, Tc='skyline', resolve_polytomies=False, max_iter=5,debug=True,time_marginal=True)
    
    tt.branch_length_to_years()
    
    treetime.treetime.treetime_to_newick(tt,root_path+'trees/Makona_600_%s.tt_dated.tree'%(gene)) ## save tree


0.00	-TreeAnc: set-up

5.71	-TreeAnc: making reduced alignment...

7.34	-TreeAnc: finished reduced alignment...
7.37	--TreeTime.reroot_to_best_root: searching for the best root position...
7.52	--TreeTime.reroot_to_best_root: branch length of children of new root: 4.503e-05, 6.097e-05

7.52	-TreeTime.reroot: with method or node: Clade
7.53	--TreeTime.reroot: Tree is being re-rooted to node new_node

7.61	-TreeAnc.optimize_sequences_and_branch_length: sequences...

7.61	-TreeAnc.infer_ancestral_sequences: method: ml
7.62	--TreeAnc._ml_anc_joint: type of reconstruction: Joint
11.41	--TreeAnc._store_compressed_sequence_pairs...

11.77	-TreeAnc.optimize_branch_length: running branch length optimization...

14.19	-TreeAnc.prune_short_branches: pruning short branches (max prob at zero)...

14.31	-TreeAnc.infer_ancestral_sequences: method: ml
14.32	--TreeAnc._ml_anc_joint: type of reconstruction: Joint
17.51	--TreeAnc._store_compressed_sequence_pairs...
17.97	--TreeAnc.optimize_sequences_and

6.87	--TreeAnc._store_compressed_sequence_pairs...

7.20	-TreeAnc.optimize_branch_length: running branch length optimization...

9.45	-TreeAnc.prune_short_branches: pruning short branches (max prob at zero)...

9.46	-TreeAnc.infer_ancestral_sequences: method: ml
9.47	--TreeAnc._ml_anc_joint: type of reconstruction: Joint
9.87	--TreeAnc._store_compressed_sequence_pairs...
9.97	--TreeAnc.optimize_sequences_and_branch_length: Iteration 1. #Nuc changed since prev reconstructions: 0
9.98	--TreeAnc.optimize_sequences_and_branch_length: Unconstrained sequence LH:-4137.403371

9.98	###TreeTime.run: INITIAL ROUND

9.98	-ClockTree: Maximum likelihood tree optimization with temporal constraints:
9.98	--ClockTree.init_date_constraints...
16.73	--ClockTime.date2dist: Setting new molecular clock. rate=7.441e-04, R^2=0.1387
16.76	--ClockTree - Joint reconstruction:  Propagating leaves -> root...
27.57	--ClockTree - Joint reconstruction:  Propagating root -> leaves...

27.66	###TreeTime.run: ITERATION

/usr/local/lib/python2.7/site-packages/treetime/distribution.py:258: RuntimeWarning: overflow encountered in exp
  return np.exp(-1 * self.__call__(x))


555.29	--ClockTree.branch_length_to_years: setting node positions in units of years


Infer mutations on a genome phylogeny.

In [4]:
root_path='/Users/evogytis/Documents/genomic-horizon/data/'

tree_path=root_path+'trees/Makona_600_genome.rooted.tree'

ll=bt.tree()
bt.make_tree(open(tree_path,'r').readline(),ll)

tip_dates={k.numName:bt.decimalDate(k.numName.split('|')[-1]) if k.numName not in masked_set else (2013.91506849,2015.91506849) for k in ll.Objects if k.branchType=='leaf'}
    
tt = treetime.TreeTime(dates=tip_dates, tree=tree_path, aln=root_path+'alignments/Makona_600_genome.fasta', gtr='HKY85')
# tt.run(root='best', infer_gtr=True, Tc='skyline', resolve_polytomies=False, max_iter=10,debug=True,time_marginal=True)
tt.infer_ancestral_sequences(method='ml',infer_gtr=True)

aln=tt.get_reconstructed_alignment()
aln_out=open('/Users/evogytis/Documents/genomic-horizon/data/alignments/Makona_600_genome.anc.fasta','w')
for seq in aln:
    aln_out.write('>%s\n%s\n'%(seq.id,seq.seq))
aln_out.close()
treetime.treetime.treetime_to_newick(tt,root_path+'trees/Makona_600_genome.anc.tree') ## clades receive different labels after ancestral state reconstruction, so we need to save it again, even though its topology is identical to the starting tree


0.00	-TreeAnc: set-up

5.84	-TreeAnc: making reduced alignment...

7.49	-TreeAnc: finished reduced alignment...

7.52	-TreeAnc.infer_ancestral_sequences: method: ml

7.52	-TreeAnc inferring the GTR model from the tree...
7.52	--TreeAnc._ml_anc_joint: type of reconstruction: Joint
11.12	--TreeAnc._store_compressed_sequence_pairs...
11.48	--TreeAnc.infer_gtr: counting mutations...

14.17	-GTR: with alphabet: ['A' 'C' 'G' 'T']

14.17	-GTR: model inference 
14.18	--TreeAnc.infer_gtr: setting overall rate to 1.0...
14.19	--TreeAnc._ml_anc_joint: type of reconstruction: Joint
17.43	--TreeAnc._store_compressed_sequence_pairs...
17.83	--TreeAnc.get_reconstructed_alignment ...


In [5]:
out_locs=open('/Users/evogytis/Documents/genomic-horizon/data/Makona_600_locs.masked.txt','w')
out_locs.write('strain,loc\n')
accession_to_strain={}

for line in open('/Users/evogytis/Documents/genomic-horizon/data/alignments/Makona_600_gp.dates.txt','r'):
    strain,date=line.strip('\n').split(',')
    loc=strain.split('|')[4]
    accession='|'.join(strain.split('|')[1:3])
    accession_to_strain[accession]=strain
    if loc=='WesternArea':
        loc='WesternUrban'
    elif loc=='':
        loc='?'
        
    if strain in masked_set:
        out_locs.write('%s,?\n'%(strain))
    else:
        out_locs.write('%s,%s\n'%(strain,loc))
    
print len(accession_to_strain)
out_locs.close()

600


In [84]:
%%bash

cd /Users/evogytis/Documents/genomic-horizon/data/trees
trees=$(ls -d *dated.tree)

locs=/Users/evogytis/Documents/genomic-horizon/data/Makona_600_locs.masked.txt

for tree in $trees
    do echo $tree
    /Users/evogytis/Documents/treetime/bin/treetime mugration --tree $tree --states $locs --confidence --attribute "loc"
    mv 2019-03-15_mugration/annotated_tree.nexus ${tree/.tt_dated/.tt_dated_traited}
    mv 2019-03-15_mugration/confidence.csv ${tree/.tt_dated.tree/.trait_confidence.txt}
done;

Makona_600_genome.tt_dated.tree

Completed mugration model inference of attribute 'loc' for Makona_600_genome.tt_dated.tree

Saved inferred mugration model as: 2019-03-15_mugration/GTR.txt
Saved table with ancestral state confidences as: 2019-03-15_mugration/confidence.csv
Saved annotated tree as: 2019-03-15_mugration/annotated_tree.nexus
Makona_600_gp.tt_dated.tree

Completed mugration model inference of attribute 'loc' for Makona_600_gp.tt_dated.tree

Saved inferred mugration model as: 2019-03-15_mugration/GTR.txt
Saved table with ancestral state confidences as: 2019-03-15_mugration/confidence.csv
Saved annotated tree as: 2019-03-15_mugration/annotated_tree.nexus


/miniconda2/lib/python2.7/site-packages/treetime/treeanc.py:1425: RuntimeWarning: divide by zero encountered in log
  node.marginal_outgroup_LH, pre = normalize_profile(np.log(node.up.marginal_profile) - node.marginal_log_Lx,


In [15]:
for gene in ['genome','gp']:
    tree_path='/Users/evogytis/Documents/genomic-horizon/data/trees/Makona_600_%s.rooted.tree'%(gene)
    
    ll=bt.tree()
    bt.make_tree(open(tree_path,'r').readline(),ll)
    ll.treeStats()


Tree height: 0.002770
Tree length: 0.068720
multitype tree
annotations present

Numbers of objects in tree: 810 (210 nodes and 600 leaves)


Tree height: 0.004500
Tree length: 0.080780
multitype tree
annotations present

Numbers of objects in tree: 642 (42 nodes and 600 leaves)

